In [38]:
# !pip install mlxtend

In [39]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import subplots as plt
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ISLP import load_data, confusion_table
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR,
      RandomForestClassifier as RC, 
      GradientBoostingClassifier as GBC)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold as KF
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      ShuffleSplit)
from sklearn.decomposition import PCA
from sklearn.base import clone
from ISLP.models import sklearn_sm
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier as KNN

# Data Engineering

In [40]:
# TumorType = pd.read_csv("labels.csv").rename(columns={"Unnamed: 0" :"Sample"})
# GeneRaw = pd.read_csv("data.csv")
# GeneRaw = GeneRaw.rename(columns={"Unnamed: 0": "Sample"})
# data = pd.merge(left=GeneRaw, right=TumorType).drop(columns={"Sample"}).rename(columns={"Class": "Tumor"})
# data
data = pd.read_csv("Cancer_Data.csv").drop(columns={"Unnamed: 32"})

# Feature Selection

In [ ]:
X = data.drop(columns={"diagnosis"})
y = data["diagnosis"]

In [ ]:
model = LogisticRegression(max_iter=500)

# forward selection to determine best features, with a cross validation of 5 folds 
# tried using 2 different sequentual feature selection tools, both take over an hour to run -- have not gotten it to finish
sfs = SFS(estimator=model, k_features='best', forward=True, scoring="recall", cv=5, n_jobs=-1)
sfs.fit(X, y)
selected_featuresLog = sfs.k_feature_names_  # Names of selected features

c:\Users\ninac\anaconda3\Lib\site-packages\mlxtend\feature_selection\sequential_feature_selector.py:800: RuntimeWarning: Mean of empty slice
  all_avg_scores.append(np.nanmean(cv_scores))


In [70]:
lsts = [[] for _ in selected_featuresLog]
for i, feature in enumerate(selected_featuresLog):
    lsts[i] = feature
log_X= data[lsts]

In [76]:
# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(log_X,
                                                    y,
                                                    random_state=314,
                                                    test_size=0.25,
                                                    shuffle=True) 

log_model = LogisticRegression(max_iter=5000)
log_model.fit(x_train, y_train)
predictions = log_model.predict(x_test)
confusion_table(predictions, y_test)

Truth,B,M
Predicted,,
B,95,48
M,0,0


Predicted Malignant = 58
Actual Malignant = 62

Predicted Benign = 130
Actual Bengin = 126

In [77]:
model2 = RC()

sfs2 = SFS(estimator=model2, k_features='best', forward=True, scoring='recall', cv=5, n_jobs=-1)
sfs2.fit(X, y)
selected_featuresRFC = sfs2.k_feature_names_  # Names of selected features
selected_featuresRFC

c:\Users\ninac\anaconda3\Lib\site-packages\mlxtend\feature_selection\sequential_feature_selector.py:800: RuntimeWarning: Mean of empty slice
  all_avg_scores.append(np.nanmean(cv_scores))


('id',)

In [79]:
lsts = [[] for _ in selected_featuresRFC]
for i, feature in enumerate(selected_featuresRFC):
    lsts[i] = feature
RFC_X= data[lsts]

# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(RFC_X,
                                                    y,
                                                    random_state=314,
                                                    test_size=0.25,
                                                    shuffle=True) 

RFC_model = RC()
RFC_model.fit(x_train, y_train)
predictions = RFC_model.predict(x_test)
confusion_table(predictions, y_test)

Truth,B,M
Predicted,,
B,64,25
M,31,23


In [80]:
model3 = KNN()

sfs3 = SFS(estimator=model3, k_features='best', forward=True, scoring='recall', cv=5)
sfs3.fit(X, y)
selected_featuresKNN = sfs3.k_feature_names_  # Names of selected features
selected_featuresKNN

c:\Users\ninac\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\ninac\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\ninac\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\ninac\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ninac\anaconda3\Lib\site-packages\sklearn\utils\_response.py", line 204, in _get_response_values
    raise ValueError(
ValueError: pos_label=1 is not a valid label: It should be one of ['B' 'M']

  warnings.warn(
c:\Users\ninac\anaconda3\Lib\site-p

('id',)

In [81]:
lsts = [[] for _ in selected_featuresRFC]
for i, feature in enumerate(selected_featuresRFC):
    lsts[i] = feature
RFC_X= data[lsts]

# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(RFC_X,
                                                    y,
                                                    random_state=314,
                                                    test_size=0.25,
                                                    shuffle=True) 

KNN_model = KNN()
RFC_model.fit(x_train, y_train)
predictions = RFC_model.predict(x_test)
confusion_table(predictions, y_test)

Truth,B,M
Predicted,,
B,64,25
M,31,23
